# MTH3302 - Méthodes probabilistes et statistiques pour I.A.
#### Polytechnique Montréal
### Projet A2024
----
Équipe T - TODO
### Objectif
Prédiction de **la consommation en carburant de voitures récentes**.

### Données
Le jeu de données contient pour presque 400 véhicule, la consommation moyenne en L/100km, l'année de frabrication, le type de véhicule, le nombre de cylindre, cylindree, la transmission et la boite.

- `train.csv` est l'ensemble d'entraînement
- `test.csv` est l'ensemble de test


## Chargement des données

Importation des librairies utilisées dans le calepin.

// TODO: à enlever à la fin:

Pour importer librairies:
using Pkg
Pkg.add(["CSV", "DataFrames", "Combinatorics", "Gadfly", "Distributions"], ...)

In [901]:
# import Pkg; Pkg.add("Plots")
# import Pkg; Pkg.add("DecisionTree")

In [902]:
# using CSV, DataFrames, Statistics, Dates, Gadfly, LinearAlgebra, Plots, DecisionTree,Random

In [903]:
function standardize(data)
    return (data .- mean(data)) ./ std(data)
end

function standardize_data(data)
    stddata = deepcopy(data)
   for col in names(stddata)
        if eltype(stddata[!, col]) <: Number && col != "id"
            stddata[!, col] = standardize(stddata[!, col])
        end
    end
    return stddata
end

standardize_data (generic function with 1 method)

In [904]:
function encode(data, column)
    for c in unique(data[!, column])
        data[!, Symbol(c)] = ifelse.(data[!, column] .== c, 1, 0)
    end
    return data
end

function encode_data(data)
    encoded_data = deepcopy(data)
    encoded_data = encode(encoded_data, :general_type)
    # encoded_data = encode(encoded_data, :annee)
    # encoded_data = encode(encoded_data, :type)
    encoded_data = encode(encoded_data, :transmission)
    encoded_data = encode(encoded_data, :boite)
    return encoded_data
end

encode_data (generic function with 1 method)

In [905]:
# faire une méthode pour les outliers

# code pas tester:
# function remove_outliers(data)
#     return data[(abs.(zscore(data)) .< 3) .| isnan.(zscore(data)), :]
# end

In [906]:
function removeRows(data)
    return select!(data, Not([:type, :transmission, :boite, :general_type, :weight, :annee]))
    # return select!(data, Not([:annee]))
end

removeRows (generic function with 1 method)

In [907]:
function add_rows(data)
    data[!,:volume_gaz] = data[!,:nombre_cylindres] .* data[!,:cylindree]

    # https://www.insurancenavy.com/average-car-weight/
    # https://www.auto-tests.com/fr/lightest-weight/Wagon/all/
    weight_dict = Dict("voiture_moyenne" => 3300, "VUS_petit" => 3500, "voiture_compacte" => 2800, "voiture_deux_places" => 2800, "voiture_minicompacte" => 1500, "VUS_standard" => 5000, "monospace" => 4500, "voiture_sous_compacte" => 2600, "camionnette_petit" => 4200, "break_petit" => 2640, "voiture_grande" => 4400, "camionnette_standard" => 4700, "break_moyen" => 3300)
    data[!, :weight] = [weight_dict[t] for t in data[!, :type]]

    general_type_dict = Dict("voiture_moyenne" => "voiture", "VUS_petit" => "VUS", "voiture_compacte" => "voiture", "voiture_deux_places" => "voiture", "voiture_minicompacte" => "voiture", "VUS_standard" => "VUS", "monospace" => "camionnette", "voiture_sous_compacte" => "voiture", "camionnette_petit" => "camionnette", "break_petit" => "break", "voiture_grande" => "voiture", "camionnette_standard" => "camionnette", "break_moyen" => "break")
    data[!, :general_type] = [general_type_dict[t] for t in data[!, :type]]
    
    # display(plot(data, x=:general_type, y=:consommation, color=:type))
    # println(combine(groupby(data, :type), nrow => :count))
    return data
end

add_rows (generic function with 1 method)

In [908]:
function getStandardEncodedData(data)
    data_copy = deepcopy(data)
    standardised_data = add_rows(data_copy)
    standardised_data = standardize_data(data_copy)
    standardised_data = encode_data(standardised_data)
    standardised_data = removeRows(standardised_data)
    
    # print(standardised_data)
    return standardised_data
end




getStandardEncodedData (generic function with 1 method)

In [909]:
function rmse(y, ychap)
    return sqrt(mean((ychap .- y).^2))
end


rmse (generic function with 1 method)

In [910]:
function rsquared(y, ychap)
    ss_total = sum((y .- mean(y)).^2)
    ss_res = sum((y .- ychap).^2)
    return ss_res / ss_total
end


rsquared (generic function with 1 method)

In [911]:
function train_test_split(data, test_size=0.2, shuffle=true)
    n = size(data, 1)
    test_size = floor(Int, n * test_size)
    
    if shuffle
        indices = randperm(n)
    else
        indices = 1:n
    end
    
    test_indices = indices[1:test_size]
    train_indices = indices[test_size+1:end]
    
    train_data = data[train_indices, :]
    test_data = data[test_indices, :]
    
    return train_data, test_data
end

train_test_split (generic function with 3 methods)

In [912]:
trainData = CSV.read("./data/train.csv", DataFrame)
testData = CSV.read("./data/test.csv", DataFrame)
trainData.consommation = parse.(Float64,replace.(trainData.consommation, "," => "."))
trainData.cylindree = parse.(Float64,replace.(trainData.cylindree, "," => "."))
testData.cylindree = parse.(Float64,replace.(testData.cylindree, "," => "."))
COMSOMMATION_MEAN = mean(trainData.consommation)
COMSOMMATION_STD = std(trainData.consommation)
trainData[!, :id] = 1:nrow(trainData)
testData[!, :id] = 1:nrow(testData)

possibles = unique(trainData.consommation)


25-element Vector{Float64}:
 13.8358823529412
  9.80041666666667
 11.7605
 13.0672222222222
  7.3503125
  7.58741935483871
 11.2004761904762
 14.700625
 12.3794736842105
 10.6913636363636
  9.4084
 15.6806666666667
  8.71148148148148
  8.11068965517241
 10.2265217391304
  7.84033333333333
  8.40035714285714
  9.04653846153846
 16.8007142857143
  6.91794117647059
  6.35702702702703
  4.52326923076923
  7.12757575757576
  4.90020833333333
  6.53361111111111

In [913]:
# function evaluate_rmse()
#     data = getStandardEncodedData(trainData)
#     nrange = 1000
#     n = 0
#     for i in range(0, 1, length=nrange)
#         n += regression(data, 0.05)[1]
#     end
#     print("average rmse for regression: ", n/nrange, "\n")

#     n = 0
#     for i in range(0, 1, length=nrange)
#         n += ridge_regression(data, 0.1, 0.05)[1]
#     end
#     print("average rmse for ridge: ", n/nrange, "\n")

#     n = 0
#     for i in range(0, 1, length=nrange)
#         n += svd_regression(data, 0.05)[1]
#     end
#     print("average rmse for svd: ", n/nrange, "\n")

#     nrange = 1000
#     n = 0
#     for i in range(0, 1, length=nrange)
#         n += polynomial_regression(data,2, 0.05)[1]
#     end
#     print("average rmse for polynomial regression: ", n/nrange, "\n")

# end


In [914]:


# function predict(data, beta)
#     return (Matrix(data[:, Not(:consommation, :id)]) * beta)
# end

# function destandardize(data)
#     return (data .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
# end

# function predict_and_destandardize(data, beta)
#     return destandardize(predict(data, beta))
# end



# function find_mistakes(data_to_predict, beta, col)
#     prediction = (Matrix(data_to_predict[:, Not(:consommation, :id)]) * beta)
#     prediction = (prediction .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
#     data_to_predict[!,:consommation] = data_to_predict[!,:consommation] .* COMSOMMATION_STD .+ COMSOMMATION_MEAN

#     idtrain = 1:size(prediction, 1)

#     base_data_of_predicted = innerjoin(data_to_predict, trainData, on=:id, makeunique=true)

#     scatter(idtrain, prediction)
#     scatter!(idtrain, base_data_of_predicted[!,:consommation])


#     difference = prediction - base_data_of_predicted[!,:consommation]
#     base_data_of_predicted[!, :difference] = difference
#     diff_cutoff = 1
#     high_diff_rows = base_data_of_predicted[abs.(difference) .> diff_cutoff, :]

#     grouped_normal = combine(groupby(base_data_of_predicted, col), nrow => :count)

#     grouped_high_diff = combine(groupby(high_diff_rows, col), nrow => :count)

#     grouped_high_diff[!, :total_diff] = combine(groupby(high_diff_rows, col), :difference => (x -> sum(abs.(x))) => :total_diff)[:, :total_diff]

#     max_diff = combine(groupby(high_diff_rows, col), :difference => (x -> maximum(abs.(x))) => :max_diff)

#     grouped_high_diff = leftjoin(grouped_high_diff, max_diff, on=col)
#     grouped_high_diff[!, :average_diff] = combine(groupby(high_diff_rows, col), :difference => (x -> mean(abs.(x))) => :average_abs_diff)[:, :average_abs_diff]

#     grouped_high_diff[!, :rmse] = combine(groupby(high_diff_rows, col), :difference => (x -> sqrt(mean(x.^2))) => :rmse)[:, :rmse]
#     grouped_normal[!, :rmse] = combine(groupby(base_data_of_predicted, col), :difference => (x -> sqrt(mean(x.^2))) => :rmse)[:, :rmse]

#     percentage_high_diff = leftjoin(grouped_high_diff, grouped_normal, on=col, makeunique=true)
#     percentage_high_diff[!, :percentage] = percentage_high_diff[!, :count] ./ percentage_high_diff[!, :count_1] .* 100
#     percentage_high_diff = sort(percentage_high_diff, :percentage, rev=true)

#     println("Percentage of data by ", col," with difference higher than 1: ")
#     println(percentage_high_diff[:, [col, :percentage, :count, :count_1, :total_diff, :max_diff, :average_diff, :rmse]])

#     grouped_normal = sort(grouped_normal, :rmse, rev=true)

#     println(grouped_normal[:, [col, :count, :rmse]])
# end




In [915]:
# rmseval, betatrain, traindata, testpredictdata = regression(data, 0.2)
# find_mistakes(testpredictdata, betatrain, :annee)

In [916]:
# beta = ridge_regression(data, 10, 0.0)[2]
# function remise_regression(beta)
#     test_data = getStandardEncodedData(testData)
#     X_test = Matrix(test_data[!,Not(:id)])
#     ychap =  X_test * beta
#     ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
#     remise(ychap)
# end
using Dates
function remise(prediction)
    id = 1:150
    df_pred = DataFrame(id=id, consommation=prediction)
    
   
    current_time = Dates.format(now(), "yyyy-mm-dd_HH-MM-SS")
    file_name = "benchmark_" * current_time * ".csv"
    CSV.write("./soumissions_potentielles/" *file_name, df_pred)
end


remise (generic function with 1 method)

In [917]:
findnearest(A::Vector{Float64},t::Float64) = A[findmin(broadcast(abs,A.-t))[2]]

function forest(data)
    train, test= train_test_split(data)

    X_train =  Matrix(train[:, Not(:consommation, :id)])
    y_train = train.consommation
    X_test = Matrix(test[:, Not(:consommation, :id)])
    y_test = test.consommation


    model = RandomForestRegressor(n_subfeatures=12, n_trees=1000,min_samples_leaf=1,min_purity_increase=0.0, max_depth=10, min_samples_split=6)
    DecisionTree.fit!(model, X_train, y_train)
    y_pred = DecisionTree.predict(model, X_test)

    train_consommation = (test.consommation * COMSOMMATION_STD) .+ COMSOMMATION_MEAN
    y_pred = (y_pred * COMSOMMATION_STD) .+ COMSOMMATION_MEAN
    #println(y_pred)
    # y_pred = map(x -> findnearest(possibles,x),y_pred)
    # println(y_pred_adjusted)
    # println(y_pred)
    # println(test.consommation)
    return sqrt(mean((y_pred - train_consommation).^2)), y_pred
end

function forest_predict(train, predict)
    X_train =  Matrix(train[:, Not(:consommation, :id)])
    y_train = train.consommation
    X_predict = Matrix(predict[:, Not(:id)])

    model = RandomForestRegressor(n_subfeatures=12, n_trees=1000,min_samples_leaf=1,min_purity_increase=0.0, max_depth=10, min_samples_split=6)
    DecisionTree.fit!(model, X_train, y_train)
    y_pred = DecisionTree.predict(model, X_predict)
    return (y_pred * COMSOMMATION_STD) .+ COMSOMMATION_MEAN
end

forest_predict (generic function with 1 method)

In [918]:
function evaluate_rmse(data)
    n = 0
    for i in 1:100
        val = forest(data)[1]
        n += val
        # println(val)
    end
    println(n/100)
end

evaluate_rmse (generic function with 1 method)

In [919]:
function main()
    trainDataStandard = getStandardEncodedData(trainData)
    predictDataStandard = getStandardEncodedData(testData)
    evaluate_rmse(trainDataStandard)
    # print(forest_predict(trainDataStandard,predictDataStandard))
end
main()

0.8302556652247886


In [920]:
# for type in unique(trainData.type)
#     println(type)
#     data_type = trainData[trainData.type .== type, :]
#     println(combine(groupby(data_type, :transmission), :consommation => mean, :volume_gaz => mean, nrow => :nrow))
#     println()
# end

In [921]:
# for type in unique(trainData.type)
#     println(type)
#     data_type = trainData[trainData.type .== type, :]
#     display(plot(x=data_type.volume_gaz, y=data_type.consommation))
#     println()
# end

## Exploration des données